In [8]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from skimage.transform import resize
import h5py

In [9]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 42x42
x_train_resized = np.zeros((x_train.shape[0], 42, 42))
for i in range(x_train.shape[0]):
    x_train_resized[i] = resize(x_train[i], (42, 42))

x_test_resized = np.zeros((x_test.shape[0], 42, 42))
for i in range(x_test.shape[0]):
    x_test_resized[i] = resize(x_test[i], (42, 42))

# Reshape to add channel dimension (required for Keras)
x_train_resized = x_train_resized.reshape(-1, 42, 42, 1)
x_test_resized = x_test_resized.reshape(-1, 42, 42, 1)

# Normalize pixel values to range [0, 1]
x_train_resized = x_train_resized.astype('float32') / 255.0
x_test_resized = x_test_resized.astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [10]:
model = Sequential()
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train_resized, y_train, batch_size=128, epochs=15, validation_data=(x_test_resized, y_test))

# Quantize the weights and biases
model_quantized = Sequential()
for layer in model.layers:
    if isinstance(layer, Dense) or isinstance(layer, Conv2D):
        weights, biases = layer.get_weights()
        weights_quantized = np.round(weights * 4) / 4  # Quantize weights to 2 bits for fixed point part
        biases_quantized = np.round(biases * 4) / 4     # Quantize biases to 2 bits for fixed point part
        layer.set_weights([weights_quantized, biases_quantized])
    model_quantized.add(layer)

# Save the quantized model to a .h5 file
model_quantized.save('quantized_mnist_model.h5')

Epoch 1/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2041 - loss: 2.2581 - val_accuracy: 0.7546 - val_loss: 1.3731
Epoch 2/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6683 - loss: 1.2082 - val_accuracy: 0.8285 - val_loss: 0.6639
Epoch 3/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7173 - loss: 0.8835 - val_accuracy: 0.8490 - val_loss: 0.5782
Epoch 4/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7415 - loss: 0.8102 - val_accuracy: 0.8571 - val_loss: 0.5364
Epoch 5/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7457 - loss: 0.7931 - val_accuracy: 0.8605 - val_loss: 0.5133
Epoch 6/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7566 - loss: 0.7680 - val_accuracy: 0.8658 - val_loss: 0.4949
Epoch 7/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7574 - loss: 0.7549 - val_accuracy: 0.8645 - val_loss: 0.5003
Epoch 8/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7634 - loss: 0.7483 - val_accuracy: 0.